# Cephノード構築：Cephノード共通初期セットアップ
Cephノード構築時の共通的な初期セットアップを行う<br>
- Caphなどパッケージインストール
- パッケージ更新
- OVSなどブリッジの初期設定
- 再起動

# パラメータ定義

## パラメータファイル設定

Cephクラスタの定義ファイルを指定するため、<br>
パラメータ定義格納ディレクトリを表示する。

In [ ]:
!ls -p ~/notebooks/share/ | grep "/$"

In [ ]:
!ls -p ~/notebooks/share/configuration

In [ ]:
!ls -p ~/notebooks/share/configuration/ceph

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "ceph_admin_vm.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/ceph/{}".format(config_file_name))
with open(config_file_path) as f:
    admin_params = yaml.safe_load(f)
admin_params

In [ ]:
target_vm_name = 'xxx-ceph-admin1'
admin_patam = admin_params[target_vm_name]
admin_patam

In [ ]:
ceph_admin_ip = admin_patam['network']['provisioning']['ip']

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "xxx-openstack-parameter.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/{}".format(config_file_name))
with open(config_file_path) as f:
    params = yaml.safe_load(f)
params

In [ ]:
repo_server = params['repository']['addresses']['service_operation']
repo_server

In [ ]:
provisioning_vip = params['openstack']['provisioning_vip']
provisioning_vip

In [ ]:
network_segment_controller_vip = params['network_segment_controller_vip']
network_segment_controller_vip

## CephノードとストレージセグメントIPアドレス対応表の読み込み

In [ ]:
import os
import csv
import pandas as pd

instance_ip_csv_path =  os.environ['HOME'] + "/notebooks/share/configuration/ceph/ceph_instance_ip.csv"
instance_ip_list = []
instance_ip_map = {}

with open(instance_ip_csv_path, 'r') as path_csv:
    reader = csv.DictReader(path_csv)
    
    for record in reader:
        instance_ip_list.append(record)
        instance_ip_map[record['instance_name']] = record

pd.DataFrame(instance_ip_list)

## 作業用ディレクトリを作成する

In [ ]:
import os,tempfile
temp_dir = tempfile.mkdtemp()
print (temp_dir)

## AnsibleのInventoryファイルを作業用ディレクトリへ作成する

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
    f.write('''[openstack_ctl]
{openstack_cmn}
'''.format(openstack_cmn = provisioning_vip))

hosts = temp_dir + "/hosts"
!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

疎通確認

In [ ]:
!ansible -m ping openstack_ctl

In [ ]:
!ansible -a 'hostname' openstack_ctl

## OpenStack用OpenRCファイル設定

`xxxxxxx`プロジェクト用のOpenRCファイルを事前にダウンロードしておくこと。  

In [ ]:
import os

user_creds = os.path.expanduser('~/.keys/xxxxxxx-openrc.sh')

assert os.path.exists(user_creds), '{} is not exist'.format(user_creds)

In [ ]:
%env USER_CREDS={user_creds}

In [ ]:
!ls -l {user_creds}

## openstackコマンド用ユーティリティ関数

In [ ]:
def build_cmdline(param):
    cmdline = ''
    
    for param_name, param_value in param.items():
        if isinstance(param_value, dict):
            for key, value in param_value.items():
                cmdline += ' --{} {}={}'.format(param_name, key, value)
        elif isinstance(param_value, list):
            for item in param_value:
                cmdline += ' --{} {}'.format(param_name, item)
        elif isinstance(param_value, bool):
            if param_value:
                cmdline += ' --{}'.format(param_name)
        else:
            cmdline += ' --{} {}'.format(param_name, param_value)

    return cmdline

In [ ]:
import json

def run_openstack(cmd):
    print('EXEC: {}'.format(cmd))
    out = !source {user_creds} && {cmd}
    print('OUTPUT: {}'.format('\n'.join(out)))
    return json.loads('\n'.join(out))

テスト

In [ ]:
run_openstack('openstack flavor list -f json' + build_cmdline({'all': True}))

## デプロイされたCephノード情報の取得

ストレージノードを特定するにあたり、ノード名称が　ceph-nodexxx （xxxの部分は、右寄せ0埋めの数字3桁）というルールに従っていることを前提としている。<br>
 例：ceph-node001<br>
 このノード名称ルールは、構築するCephクラスタ毎に変えるため、以下で定義する。

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

In [ ]:
import pandas as pd
import re

name_match_rule = r'ceph-xxx-node-[0-9]+'

## OpenStack Ironicからノード情報の取得

In [ ]:
cmd = "openstack baremetal node list -f json --long"
nodes = run_openstack(cmd)

In [ ]:
import pandas as pd
import re

for node in nodes:
    instance_uuid = node.get("Instance UUID", None)
    if instance_uuid is None:
        continue
        
    node["instance_name"] = node["Instance Info"]["display_name"]

pd.set_option('display.max_rows', None)
pd.DataFrame(nodes, columns=['Instance UUID', 'instance_name', 'UUID', 'Maintenance', 'Provisioning State', 'Power State', 'Name'])

In [ ]:
cmd = "openstack server list -f json --long"
nova_server_map = run_openstack(cmd)

In [ ]:
node_list = []
for server in nova_server_map:
    node_id = server['ID']
    if re.match(name_match_rule, server['Name']) is None:
        continue
    for node in nodes:
        if node['Instance UUID'] == node_id:
            node_info = {}
            node_list.append(node_info)
            node_info['name'] = server['Name']
            node_info['node_name'] = node['Name']
            instance_ip_info = instance_ip_map.get(server['Name'], None)
            if instance_ip_info is not None:
                node_info['instance_ip'] = instance_ip_info.get('ipv4', None)
            else:
                node_info['instance_ip'] = None
            address = server['Networks']
            prov_net = address['provisioning-net']
            for addr in prov_net:
                node_info['prov_ip'] = addr

pd.DataFrame(node_list)

## 初期化対象Cephノード情報の取得

初期化対象のインスタンス名称を設定する

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

### 初期化対象ノードの指定

In [ ]:
instance_name = 'ceph-xxx-node-001'

In [ ]:
target = None
for elem in node_list:
    if elem['name'] == instance_name:
        target = elem
        break

assert (target is not None), "指定されたターゲットノードが存在しません。"
print (target)

## 一時Inventoryファイルの生成
以下を実行し、作業用ディレクトリ配下に一時Inventoryファイルを生成する。

Inventoryファイルを生成する。

In [ ]:
node_user='xxxxx'
proxy_user='xxxxx'
proxy_key='~/.ssh/id_ras'

In [ ]:
ceph_node = target["prov_ip"]
ceph_node_strage_ip = target.get("instance_ip",None)
strage_prefix = "16"

print ("ceph_node={} ceph_node_strage_ip={}".format(ceph_node, ceph_node_strage_ip))

sudoでパスワードが必要な場合は、パスワード入力を設定してansible.cfgに設定すること。

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
    f.write('''[openstack_ctl]
{openstack_cmn}
'''.format(openstack_cmn = provisioning_vip))
    f.write('''[ceph_admin]
{ceph_admin_ip}   ansible_user={proxy_user} ansible_ssh_private_key_file={proxy_key}  ansible_python_interpreter=/usr/bin/python3
[ceph_node]
{node_ip}  ansible_user={node_user} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"'  ansible_python_interpreter=/usr/bin/python3
[repo_server]
{repo_server}
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=ceph_node, repo_server=repo_server,
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))

hosts = temp_dir + "/hosts"

!cat $hosts

## 疎通確認
以下コマンドを実行し、OpeHubからCephAdmin経由で、Cephノードに疎通することを確認する。

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' ceph_admin

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' repo_server

# Timezone 設定
TimezoneをUTCに変更する

In [ ]:
!ansible -i $hosts -b -m shell -a 'timedatectl list-timezones | grep UTC' ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a 'timedatectl set-timezone UTC' ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a 'date' ceph_node

# リポジトリ設定

## リポジトリ定義
リポジトリ設定を定義する。<br>
以下コマンドより、世代リポジトリの一覧を表示する。<br>
一覧から、必要なリポジトリを選択し、ディレクトリ名(日付(yyyymmdd)_時間(HHMM)」を選択し、「REPO_GENERATION」に入力すること。

### リポジトリ確認

DocumentRootは`/repository`である。

In [ ]:
document_root='/repository'

DocumentRoot配下を確認する。

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la $document_root" repo_server

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la $document_root/20240520" repo_server

### CentOS9 streamリポジトリ定義

In [ ]:
# 世代リポジトリ
REPO_GENERATION="xxxxxxxx"

.repoファイルを一時ディレクトリに作成する。

In [ ]:
!mkdir -p $temp_dir/etc/yum.repos.d/

In [ ]:
!echo $REPO_GENERATION

In [ ]:
import os
with open(os.path.join(temp_dir, 'etc/yum.repos.d/centos.repo'), 'w') as f:
    f.write('''[baseos]
name=CentOS Stream $releasever - BaseOS
baseurl=http://{repo_ip}/{nlink}/baseos
gpgcheck=0
enabled=1

[appstream]
name=CentOS Stream $releasever - AppStream
baseurl=http://{repo_ip}/{nlink}/appstream
gpgcheck=0
enabled=1
'''.format(repo_ip=repo_server,
           nlink=REPO_GENERATION))

!cat $temp_dir/etc/yum.repos.d/centos.repo

In [ ]:
with open(os.path.join(temp_dir, 'etc/yum.repos.d/centos-addons.repo'), 'w') as f:
    f.write('''[highavailability]
name=CentOS Stream $releasever - HighAvailability
baseurl=http://{repo_ip}/{nlink}/highavailability
gpgcheck=0
enabled=1

[nfv]
name=CentOS Stream $releasever - NFV
baseurl=http://{repo_ip}/{nlink}/nfv
gpgcheck=0
enabled=1

[resilientstorage]
name=CentOS Stream $releasever - ResilientStorage
baseurl=http://{repo_ip}/{nlink}/resilientstorage
gpgcheck=0
enabled=1

[extras-common]
name=CentOS Stream $releasever - Extras packages
baseurl=http://{repo_ip}/{nlink}/extras-common
gpgcheck=0
enabled=1
'''.format(repo_ip=repo_server,
           nlink=REPO_GENERATION))

!cat $temp_dir/etc/yum.repos.d/centos-addons.repo

### epelリポジトリ定義

In [ ]:
with open(os.path.join(temp_dir, 'etc/yum.repos.d/epel.repo'), 'w') as f:
    f.write('''[epel]
#name=Extra Packages for Enterprise Linux $releasever - $basearch
# It is much more secure to use the metalink, but if you wish to use a local mirror
# place its address here.
#baseurl=http://download.example/pub/epel/$releasever/Everything/$basearch/
baseurl=http://{repo_ip}/{nlink}/epel/
enabled=1
gpgcheck=0
#countme=1
'''.format(repo_ip=repo_server,
           nlink=REPO_GENERATION))

!cat $temp_dir/etc/yum.repos.d/epel.repo

### epel-nextリポジトリ設定

In [ ]:
with open(os.path.join(temp_dir, 'etc/yum.repos.d/epel-next.repo'), 'w') as f:
    f.write('''[epel-next]
#name=Extra Packages for Enterprise Linux $releasever - $basearch
# It is much more secure to use the metalink, but if you wish to use a local mirror
# place its address here.
#baseurl=http://download.example/pub/epel/$releasever/Everything/$basearch/
baseurl=http://{repo_ip}/{nlink}/epel-next/
enabled=1
gpgcheck=0
#countme=1
'''.format(repo_ip=repo_server,
           nlink=REPO_GENERATION))

!cat $temp_dir/etc/yum.repos.d/epel-next.repo

### Cephリポジトリ定義

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la $document_root" repo_server

In [ ]:
# 世代リポジトリ
REPO_GENERATION="xxxxxxxx"

In [ ]:
!echo $REPO_GENERATION

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la $document_root/$REPO_GENERATION" repo_server

In [ ]:
with open(os.path.join(temp_dir, 'etc/yum.repos.d/centos-ceph-quincy.repo'), 'w') as f:
    f.write('''[centos-ceph-quincy]
name=CentOS-$releasever - Ceph Quincy
baseurl=http://{repo_ip}/{nlink}/centos-ceph-quincy
gpgcheck=0
enabled=1
'''.format(repo_ip=repo_server,
           nlink=REPO_GENERATION))

!cat $temp_dir/etc/yum.repos.d/centos-ceph-quincy.repo

In [ ]:
with open(os.path.join(temp_dir, 'etc/yum.repos.d/centos-nfv-openvswitch.repo'), 'w') as f:
    f.write('''[centos-nfv-openvswitch.repo]
name=CentOS-$releasever - NFV OpenvSwitch
baseurl=http://{repo_ip}/{nlink}/centos-nfv-openvswitch
gpgcheck=0
enabled=1
'''.format(repo_ip=repo_server,
           nlink=REPO_GENERATION))

!cat $temp_dir/etc/yum.repos.d/centos-nfv-openvswitch.repo

## リポジトリファイル転送

### 元のリポジトリファイルの無効化

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la /etc/yum.repos.d" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/centos.repo /etc/yum.repos.d/centos.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/centos-addons.repo /etc/yum.repos.d/centos-addons.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/epel-cisco-openh264.repo /etc/yum.repos.d/epel-cisco-openh264.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/epel-next.repo /etc/yum.repos.d/epel-next.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/epel-next-testing.repo /etc/yum.repos.d/epel-next-testing.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/epel.repo /etc/yum.repos.d/epel.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/epel-testing.repo /etc/yum.repos.d/epel-testing.repo.rpmsave" ceph_node

In [ ]:
#!ansible -i $hosts -b -m shell -a "mv /etc/yum.repos.d/td.repo /etc/yum.repos.d/td.repo.rpmsave" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la /etc/yum.repos.d" ceph_node

### リポジトリファイル転送

In [ ]:
!ansible -i $hosts -b -m copy -a "src=$temp_dir/etc/yum.repos.d/centos.repo dest=/etc/yum.repos.d/centos.repo" ceph_node

In [ ]:
!ansible -i $hosts -b -m copy -a "src=$temp_dir/etc/yum.repos.d/centos-addons.repo dest=/etc/yum.repos.d/centos-addons.repo" ceph_node

In [ ]:
!ansible -i $hosts -b -m copy -a "src=$temp_dir/etc/yum.repos.d/centos-ceph-quincy.repo dest=/etc/yum.repos.d/centos-ceph-quincy.repo" ceph_node

In [ ]:
!ansible -i $hosts -b -m copy -a "src=$temp_dir/etc/yum.repos.d/centos-nfv-openvswitch.repo dest=/etc/yum.repos.d/centos-nfv-openvswitch.repo" ceph_node

In [ ]:
!ansible -i $hosts -b -m copy -a "src=$temp_dir/etc/yum.repos.d/epel.repo dest=/etc/yum.repos.d/epel.repo" ceph_node

In [ ]:
!ansible -i $hosts -b -m copy -a "src=$temp_dir/etc/yum.repos.d/epel-next.repo dest=/etc/yum.repos.d/epel-next.repo" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ls -la /etc/yum.repos.d" ceph_node

## dnf proxy設定

以下の手順で、CephAdminをProxyとして設定する。

In [ ]:
!ansible -i $hosts -m shell -a 'cat /etc/dnf/dnf.conf' ceph_node

In [ ]:
!ansible -i $hosts -b -m ini_file -a "dest=/etc/dnf/dnf.conf section=main option=proxy value=http://{ceph_admin_ip}:8080 state=present" ceph_node

In [ ]:
!ansible -i $hosts -m shell -a 'cat /etc/dnf/dnf.conf' ceph_node

Proxy疎通確認

In [ ]:
!ansible -i $hosts -b -m dnf -a "list='ceph'" ceph_node

In [ ]:
!ansible -i $hosts -b -m dnf -a "list='openvswitch*'" ceph_node

In [ ]:
!ansible -i $hosts -b -m dnf -a "list='python3-pbr'" ceph_node

## リポジトリ更新とアップデート

In [ ]:
!ansible -i $hosts -b -m shell -a "dnf clean all --enablerepo='*'" ceph_node

In [ ]:
!ansible -i $hosts -b -m dnf -a "name='*' state=latest" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "dnf list openvswitch*" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "dnf list ceph*" ceph_node

In [ ]:
!ansible -i $hosts -b -m reboot -a "reboot_timeout=3600" ceph_node

# Cephパッケージインストール
CephNodeに必要なパッケージをインストールする。<br>

## playbook作成


In [ ]:
!mkdir -p $temp_dir/roles/ceph_packages/tasks

In [ ]:
%%writefile $temp_dir/roles/ceph_packages/tasks/main.yml
- name: install required packages
  dnf:
    name: 
      - cephadm
      - ceph
      - python3
      - python3-flask
      - python3-requests
      - python3-pbr
      - python3-netifaces
      - libcurl-devel
      - NetworkManager-ovs
      - openvswitch2.17
      - lvm2     

In [ ]:
!tree $temp_dir/roles

In [ ]:
import os
with open(os.path.join(temp_dir, 'ceph_node_packages.yml'), 'w') as f:
    f.write('''- hosts: {hosts}
  roles:
    - role: ceph_packages
'''.format(hosts=ceph_node))


## dry-run

check modeで動作させ、結果がok=2であることを確認する。

In [ ]:
result = !ansible-playbook -b -i $hosts -CDv $temp_dir/ceph_node_packages.yml
result

In [ ]:
if not (result[-2].find('ok=2') > -1):
    raise Exception

## run

変更対象が意図したホストであることを確認して、実行する。

In [ ]:
!ansible-playbook -b -i $hosts $temp_dir/ceph_node_packages.yml

# Chrony（NTP）セットアップ

In [ ]:
!ansible -i $hosts -b -m shell -a "systemctl status chronyd" $ceph_node

## chrony.confの設定

In [ ]:
ntp_servers = []
for key,value in admin_params.items():
    if type(value) is not dict:
        continue
    ip = value['network']['provisioning']['ip']
    ntp_servers.append(ip)

print(ntp_servers)

In [ ]:
ntp_servers_conf = '\n'.join(['server {} iburst'.format(x) for x in ntp_servers])
print (ntp_servers_conf)

In [ ]:
chrony_conf = '''# Use public servers from the pool.ntp.org project.
# Please consider joining the pool (http://www.pool.ntp.org/join.html).
#server 0.centos.pool.ntp.org iburst
#server 1.centos.pool.ntp.org iburst
#server 2.centos.pool.ntp.org iburst
#server 3.centos.pool.ntp.org iburst
{servers}

# Ignore stratum in source selection.
stratumweight 0

# Record the rate at which the system clock gains/losses time.
driftfile /var/lib/chrony/drift

# Enable kernel RTC synchronization.
rtcsync

# In first three updates step the system clock instead of slew
# if the adjustment is larger than 10 seconds.
makestep 10 3

# Allow NTP client access from local network.
#allow 192.168/16

# Listen for commands only on localhost.
bindcmdaddress 127.0.0.1
bindcmdaddress ::1

# Serve time even if not synchronized to any NTP server.
#local stratum 10

keyfile /etc/chrony.keys

# Specify the key used as password for chronyc.
commandkey 1

# Generate command key if missing.
generatecommandkey

# Disable logging of client accesses.
noclientlog

# Send a message to syslog if a clock adjustment is larger than 0.5 seconds.
logchange 0.5

logdir /var/log/chrony
#log measurements statistics tracking
'''.format(servers=ntp_servers_conf)

import os.path

with open(os.path.join(temp_dir, 'chrony.conf'), 'w') as f:
    f.write(chrony_conf)

!cat {temp_dir}/chrony.conf

In [ ]:
!ansible --check --diff -i $hosts -b -m copy -a "dest=/etc/chrony.conf src={temp_dir}/chrony.conf backup=yes" $ceph_node

In [ ]:
!ansible --diff -i $hosts -b -m copy -a "dest=/etc/chrony.conf src={temp_dir}/chrony.conf backup=yes" $ceph_node

In [ ]:
!ansible -i $hosts -m service -b -a "name=chronyd enabled=yes state=restarted" $ceph_node

時刻同期確認

In [ ]:
!ansible -i $hosts -b -m shell -a "chronyc sources" $ceph_node

# firewalld 設定

In [ ]:
!ansible -i $hosts -b -m dnf -a "name=firewalld state=latest" ceph_node

In [ ]:
!ansible -i $hosts -m service -b -a "name=firewalld enabled=yes state=started" ceph_node

## firewalld チェック

In [ ]:
!ansible -i $hosts -b -m shell -a "firewall-cmd --list-all" ceph_node

## firewalldセットアップ

### MON OSD

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=6800-7300 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=6789 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=3300 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=6800-7300 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=6789 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=3300 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### ovs-ctrl API

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=7301 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### node_exporter ポート

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=9100 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=9100 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=9100 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### prometheus ポート

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=9095 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=9095 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### alertmanager

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=9093 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=9093 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=9093 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=9094 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=9094 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=9094 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### grafana

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=3000 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=3000 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### dashboard

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=8443 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=8443 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### Ceph Manager Prometheus エクスポーターモジュール

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=9283 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=9283 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=9283 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### 	Ceph iSCSI ゲートウェイデータ

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.yy.0.0/16 port port=9287 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=9287 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/24 port port=9287 protocol=tcp accept' permanent=true state=enabled immediate=yes" ceph_node

### ssh用ポート解放

In [ ]:
!ansible -i $hosts -b -m firewalld -a "service=ssh permanent=true state=disabled" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=xx.zz.0.0/16 port port=22 protocol=tcp accept' permanent=true state=enabled" ceph_node

In [ ]:
!ansible -i $hosts -b -m firewalld -a "rich_rule='rule family=ipv4 source address=aa.bb.cc.0/16 port port=22 protocol=tcp accept' permanent=true state=enabled" ceph_node

### firewalld再起動

In [ ]:
!ansible -i $hosts -m service -b -a "name=firewalld enabled=yes state=restarted" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "firewall-cmd --list-all" ceph_node

# ovs有効化

In [ ]:
!ansible -i $hosts -b -m service -a "name=openvswitch enabled=yes state=started" ceph_node

## ovs動作確認

In [ ]:
!ansible -i $hosts -b -m shell -a "ovs-vsctl show" ceph_node

# 更新と再起動

- OS,パッケージの更新を実施
- OS更新を反映させるためノードの再起動

In [ ]:
!ansible -i $hosts -b -m dnf -a "name='*' state=latest" ceph_node

In [ ]:
!ansible -i $hosts -b -m reboot -a "reboot_timeout=3600" ceph_node

# StrageSegment OVSブリッジ設定
StrageSegmentに接続されているインタフェースを確認し、bonding接続したOVSブリッジの設定を行う。<br>
StrageSegmentに接続されているR740のインタフェースは、スロット3の2番と、スロット4の1番となっている。

In [ ]:
print("ceph_node_strage_ip={}".format(ceph_node_strage_ip))

## インタフェース状態確認

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn show" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli d" ceph_node

In [ ]:
# StrageSegmentに接続されているR740/750のbonding対象インタフェース名称を確認する
raise Exception('この先手作業が必要です')

In [ ]:
# StrageSegmentに接続されているR740/750のbonding対象インタフェース名称を定義する
interface_list = ["ens3f1","ens4f1"]

bounding_if = "strage_bond"
strage_bridge = "br-ceph"
loca_if = "br-ceph-local"

## OVSブリッジ生成

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn add type ovs-bridge conn.interface {strage_bridge}" ceph_node

In [ ]:
!sleep 5

## Bonding インタフェース生成

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn add type ovs-port conn.interface port1 master {strage_bridge}" ceph_node

In [ ]:
!sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn add type bond con-name {bounding_if} ifname {bounding_if} mode 802.3ad master port1" ceph_node

In [ ]:
!sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn modify {bounding_if} 802-3.mtu 9000" ceph_node

In [ ]:
!sleep 5

In [ ]:
for if_name in interface_list:
    !ansible -i $hosts -b -m shell -a "nmcli conn add type ethernet slave-type bond con-name {bounding_if}-{if_name} ifname {if_name} master {bounding_if}" ceph_node
    !sleep 5

In [ ]:
for if_name in interface_list:
    !ansible -i $hosts -b -m shell -a "nmcli conn modify {bounding_if}-{if_name} 802-3.mtu 9000" ceph_node
    !sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn down {bounding_if}" ceph_node

In [ ]:
!sleep 1

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn up {bounding_if}" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ip a" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli d show {strage_bridge}" ceph_node

In [ ]:
for if_name in interface_list:
    !ansible -i $hosts -b -m shell -a "nmcli d show {if_name}" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli d show {bounding_if}" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli d show port1" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ovs-vsctl show" ceph_node

## StrageSegment側のIPアドレスの設定

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn add type ovs-port conn.interface port2 master {strage_bridge}" ceph_node

In [ ]:
!sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn add type ovs-interface slave-type ovs-port conn.interface {loca_if} con-name {loca_if} master port2" ceph_node

In [ ]:
!sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli connection modify {loca_if} 802-3.mtu 9000" ceph_node

In [ ]:
!sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli con mod {loca_if} ipv6.method ignore" ceph_node

In [ ]:
!sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli con mod {loca_if} ipv4.may-fail no" ceph_node

In [ ]:
!sleep 5

In [ ]:
if ceph_node_strage_ip is not None:
    !ansible -i $hosts -b -m shell -a "nmcli con mod {loca_if} ipv4.addresses '{ceph_node_strage_ip}/{strage_prefix}'" ceph_node
    !sleep 5

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli con mod {loca_if} ipv4.method manual" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli d show {loca_if}" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli con show {loca_if}" ceph_node

## 再起動によるネットワーク設定更新と確認

In [ ]:
!ansible -i $hosts -b -m reboot -a "reboot_timeout=3600" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "nmcli conn" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ip a" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ovs-vsctl show" ceph_node

In [ ]:
!ansible -i $hosts -b -m shell -a "ovs-ofctl show {strage_bridge}" ceph_node

# 後始末

一時ディレクトリを削除する。

In [ ]:
!rm -fr $temp_dir